In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/gb_cities.csv')
coordinates = df[['Longitude', 'Latitude']].values
names = df['Place Name'].values

In [ ]:
!pip install routingpy
import routingpy as rp
import numpy as np

# get a free key at https://www.graphhopper.com/
api_key = "84df0674-ccf1-44c4-9f65-c981722a42e7"
api = rp.Graphhopper(api_key=api_key)
matrix = api.matrix(locations=coordinates, profile='car')
durations = np.matrix(matrix.durations)
print(durations)

[[    0 10892 30363 ... 23368 25227 19833]
 [10901     0 23625 ... 16458 18317 13095]
 [30326 23541     0 ...  8835  9441 12260]
 ...
 [23394 16444  9007 ...     0  2789  7924]
 [25274 18324  9654 ...  2857     0  9627]
 [19854 13069 12340 ...  8002  9637     0]]


In [ ]:
def symmetricize(m, high_int=None):

    # if high_int not provided, make it equal to 10 times the max value:
    if high_int is None:
        high_int = round(10*m.max())

    m_bar = m.copy()
    np.fill_diagonal(m_bar, 0)
    u = np.matrix(np.ones(m.shape) * high_int)
    np.fill_diagonal(u, 0)
    m_symm_top = np.concatenate((u, np.transpose(m_bar)), axis=1)
    m_symm_bottom = np.concatenate((m_bar, u), axis=1)
    m_symm = np.concatenate((m_symm_top, m_symm_bottom), axis=0)

    return m_symm.astype(int) # Concorde requires integer weights


In [ ]:
symmetricize(durations)

matrix([[     0, 470690, 470690, ...,  23394,  25274,  19854],
        [470690,      0, 470690, ...,  16444,  18324,  13069],
        [470690, 470690,      0, ...,   9007,   9654,  12340],
        ...,
        [ 23394,  16444,   9007, ...,      0, 470690, 470690],
        [ 25274,  18324,   9654, ..., 470690,      0, 470690],
        [ 19854,  13069,  12340, ..., 470690, 470690,      0]])

In [ ]:
!git clone https://github.com/jvkersch/pyconcorde

Cloning into 'pyconcorde'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (408/408), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 408 (delta 199), reused 364 (delta 176), pack-reused 0
Receiving objects: 100% (408/408), 3.54 MiB | 17.50 MiB/s, done.
Resolving deltas: 100% (199/199), done.


In [ ]:
%cd pyconcorde

/content/pyconcorde


In [ ]:
!pip install -e .

Obtaining file:///content/pyconcorde
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.4 MB/s eta 0:00:00
  Building editable for pyconcorde (pyproject.toml) ... done
  Created wheel for pyconcorde: filename=pyconcorde-0.1.0-0.editable-cp310-cp310-linux_x86_64.whl size=3557 sha256=cb2e2458b8aaaa5789a7e738d70876057c0c27bd86ae42327894d8c5566f294d
  Stored in directory: /tmp/pip-ephem-wheel-cache-dykaaz_t/wheels/30/8f/f8/baf479acd96e33ee68ba2acd29c1f1ae051ed8eaddc8bcd594
Successfully built pyconcorde
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3

In [ ]:
from concorde.problem import Problem
from concorde.concorde import Concorde

def solve_concorde(matrix):
    problem = Problem.from_matrix(matrix)
    solver = Concorde()
    solution = solver.solve(problem)
    print(f'Optimal tour: {solution.tour}')
    return solution

In [ ]:
durations_symm = symmetricize(durations)
solution = solve_concorde(durations_symm)

Optimal tour: [0, 79, 22, 101, 25, 104, 48, 127, 68, 147, 23, 102, 58, 137, 7, 86, 39, 118, 73, 152, 78, 157, 36, 115, 42, 121, 62, 141, 16, 95, 20, 99, 51, 130, 40, 119, 19, 98, 59, 138, 50, 129, 54, 133, 27, 106, 10, 89, 4, 83, 66, 145, 33, 112, 14, 93, 2, 81, 45, 124, 32, 111, 11, 90, 29, 108, 34, 113, 24, 103, 8, 87, 17, 96, 56, 135, 64, 143, 61, 140, 75, 154, 52, 131, 71, 150, 18, 97, 3, 82, 9, 88, 74, 153, 55, 134, 72, 151, 28, 107, 12, 91, 70, 149, 65, 144, 35, 114, 31, 110, 77, 156, 63, 142, 41, 120, 69, 148, 6, 85, 76, 155, 67, 146, 15, 94, 44, 123, 47, 126, 60, 139, 57, 136, 38, 117, 13, 92, 5, 84, 43, 122, 49, 128, 46, 125, 21, 100, 1, 80, 30, 109, 53, 132, 37, 116, 26, 105]


In [ ]:
# pick alternate elements: these correspond to the originals
tour = solution.tour[::2]

# order the original coordinates and names
coords_ordered = [coordinates[i].tolist() for i in tour]
names_ordered = [names[i] for i in tour]

In [ ]:
# add back in the first for a complete loop
coords_ordered_return = coords_ordered + [coords_ordered[0]]

# obtain complete driving directions for the ordered loop
directions = api.directions(locations=coords_ordered_return, profile='car')

In [15]:
import folium
def generate_map(coordinates, names, directions):

    # folium needs lat, long
    coordinates = [(y, x) for (x, y) in coordinates]
    route_points = [(y, x) for (x, y) in directions.geometry]
    lat_centre = np.mean([x for (x, y) in coordinates])
    lon_centre = np.mean([y for (x, y) in coordinates])
    centre = lat_centre, lon_centre

    m = folium.Map(location=centre, zoom_start=1, zoom_control=False)

    # plot the route line
    folium.PolyLine(route_points, color='red', weight=2).add_to(m)

    # plot each point with a hover tooltip
    for i, (point, name) in enumerate(zip(coordinates, names)):
        folium.CircleMarker(location=point,
                      tooltip=f'{i}: {name}',
                      radius=2).add_to(m)

    custom_tile_layer = folium.TileLayer(
        tiles='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png',
        attr='CartoDB Positron',
        name='Positron',
        overlay=True,
        control=True,
        opacity=0.7  # Adjust opacity to control the level of greying out
    )

    custom_tile_layer.add_to(m)
    folium.LayerControl().add_to(m)

    sw = (np.min([x for (x, y) in coordinates]), np.min([y for (x, y) in coordinates]))
    ne = (np.max([x for (x, y) in coordinates]), np.max([y for (x, y) in coordinates]))
    m.fit_bounds([sw, ne])

    return m

generate_map(coords_ordered, names_ordered, directions).save('/content/drive/MyDrive/Colab_Notebooks/gb_cities.html')

In [17]:
def generate_gpx_file(directions, filename):
    gpx_template = """<?xml version="1.0" encoding="UTF-8"?>
    <gpx version="1.1" xmlns="http://www.topografix.com/GPX/1/1"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://www.topografix.com/GPX/1/1
        http://www.topografix.com/GPX/1/1/gpx.xsd">
        <trk>
            <name>Track</name>
            <trkseg>{}</trkseg>
        </trk>
    </gpx>
    """

    trkseg_template = """
        <trkpt lat="{}" lon="{}"/>
    """

    trkseg_elements = ""
    for point in directions.geometry:
        trkseg_elements += trkseg_template.format(point[1], point[0])

    gpx_data = gpx_template.format(trkseg_elements)

    with open(filename, 'w') as file:
        file.write(gpx_data)

generate_gpx_file(directions, '/content/drive/MyDrive/Colab_Notebooks/gb_cities.gpx')